In [1]:
# Based on code from https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

import nltk
import spacy
import gensim
import random
import pickle
import pandas as pd
from gensim import corpora
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

spacy.load('en_core_web_sm')
#spacy.load('es_core_news_sm')
#spacy.load("fr_core_news_sm")
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
#en_stop = set(nltk.corpus.stopwords.words('spanish'))
#en_stop = set(nltk.corpus.stopwords.words('french'))

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to /Users/fv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/fv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_topics(path, num_topics):
    text_data = []
    f = pd.read_csv(path, 'rb', delimiter = '\t')
    for line in f['text']:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            #print('List of tokens:', tokens)
            text_data.append(tokens)

    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]

    pickle.dump(corpus, open('models/corpus.pkl', 'wb'))
    dictionary.save('models/dictionary.gensim')

    NUM_TOPICS = num_topics
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('models/model' + str(NUM_TOPICS) + '.gensim')
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

In [31]:
get_topics('data/out/IHorgs/a14_who/a14_who.csv', 5)

(0, '0.063*"covid19" + 0.063*"coronavirus" + 0.034*"whorajray" + 0.034*"nitaambani" + 0.034*"mukeshambani" + 0.034*"narendramodi" + 0.034*"indiafightscorona" + 0.034*"rajray" + 0.034*"111/138" + 0.034*"everybreathcounts"')
(1, '0.048*"covid19" + 0.048*"trump" + 0.048*"propangandized" + 0.048*"virus" + 0.048*"death" + 0.048*"distance" + 0.048*"canada" + 0.048*"stewardship" + 0.048*"dither" + 0.048*"deadly"')
(2, '0.043*"disponible" + 0.043*"pandemia" + 0.043*"combatir" + 0.043*"diseñada" + 0.043*"lanzamiento" + 0.043*"whoacademy" + 0.043*"vidas" + 0.043*"ampliar" + 0.043*"trabajadores" + 0.043*"permitir"')
(3, '0.048*"tecikorodu" + 0.048*"precaution" + 0.048*"healing" + 0.048*"adhere" + 0.048*"people" + 0.048*"trust" + 0.048*"recover" + 0.048*"encourage" + 0.048*"safety" + 0.048*"beyondordinary"')
(4, '0.017*"covid19" + 0.017*"indiafightscorona" + 0.017*"rajray" + 0.017*"mukeshambani" + 0.017*"whorajray" + 0.017*"nitaambani" + 0.017*"narendramodi" + 0.017*"coronavirus" + 0.017*"confirm"